In [7]:
from biosyn.dataloader import load_dictionary, load_queries
from transformers import AutoModel, AutoTokenizer
import torch
TRAIN_DICT_PATH = "./data/data-ncbi-fair/train_dictionary.txt"
TRAIN_DIR = "./data/data-ncbi-fair/traindev"

train_dictionary  = load_dictionary(dict_path=TRAIN_DICT_PATH)
train_queries  = load_queries(data_dir=TRAIN_DIR, filter_composite=False, filter_duplicates=False, filter_cuiless=True)

# train_dictionary = train_dictionary[:50]
# train_queries = train_queries[:10]

# tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
# encoder = AutoModel.from_pretrained('dmis-lab/biobert-base-cased-v1.1')


max_length = 25
topk = 20

query_names, query_ids = [row[0] for row in train_queries], [row[1] for row in train_queries]
dict_names, dict_ids = [row[0] for row in train_dictionary], [row[1] for row in train_dictionary]

100%|██████████| 691/691 [00:00<00:00, 13927.33it/s]


In [8]:
import pickle 
def save_pkl(ar, fp):
    with open(fp, 'wb') as f:
        pickle.dump(ar, f)
def get_pkl(fp):
    with open(fp, "rb") as f:
        return pickle.load(f)

In [9]:
save_pkl(train_queries, "./data/pkls/queries.pkl")
save_pkl(train_dictionary, "./data/pkls/dict.pkl")

In [2]:
from biosyn.biosyn import BioSyn
from biosyn.dataloader import NamesDataset

model_name_or_path = 'dmis-lab/biobert-base-cased-v1.1'
biosyn = BioSyn(max_length, False, topk)

biosyn.load_dense_encoder(model_name_or_path)
len(dict_names)

50000

In [ ]:
# batch_size = 64
# names = dict_names
# name_tokens = biosyn.tokenizer(names,
#                 padding="max_length",
#                 max_length=max_length,
#                 truncation=True,
#                 return_tensors="pt"
#             )
# name_dataset = NamesDataset(name_tokens)
# from torch.utils.data import DataLoader
# from transformers import default_data_collator
# data_loader = DataLoader(
#                 name_dataset,
#                 shuffle=False,
#                 collate_fn=default_data_collator,
#                 batch_size=batch_size
#             )
# N = len(names)
# H = getattr(getattr(biosyn.encoder, "config", None), "hidden_size", None)
# N, H
# out_cpu = torch.empty((N,H), dtype=torch.float32, pin_memory=True)


In [3]:
dict_embs = biosyn.build_faiss_index(dict_names=dict_names)


KeyboardInterrupt: 